In [4]:
import copy
import pygame
import random
import pyomo.environ as pyo
from dataclasses import dataclass

# Pygame initialization and settings
pygame.init()
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
WHITE = (240, 240, 240)
GREEN = (0, 200, 0)


screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
transform = lambda x, y: (x, SCREEN_HEIGHT - y)




# Model parameters
N = 10  # Prediction horizon
fps = 30
dt = 1 / fps
gravity = -50

    
# Create a Pyomo model for MPC
model = pyo.ConcreteModel()
model.k = pyo.RangeSet(1, N)
model.u = pyo.Var(model.k, within=pyo.Reals, bounds=(40, 60))
model.T_target = pyo.Param(initialize=300.0, mutable=True)


def calculate_the_control_signal(bird: Bird, pipe: Pipe) -> float:
    # Delete old model components if they exist
    for v in ['y', 'vy', 'dynamic_constraint_y', 'dynamic_constraint_vy', 'objective']:
        if hasattr(model, v):
            model.del_component(getattr(model, v))

    model.y = pyo.Var(model.k, initialize=bird.y)  # Bird position y
    model.vy = pyo.Var(model.k, initialize=bird.vy)  # Bird velocity y

    # Update the target position based on the pipe position
    model.T_target = pipe.h + pipe.gap / 2

    # Dynamics constraints for the bird's vertical position and velocity
    def dynamic_constraint_y(model, k):
        if k == 1:
            return model.y[k] == bird.y + bird.vy * dt
        else:
            return model.y[k] == model.y[k-1] + model.vy[k-1] * dt

    def dynamic_constraint_vy(model, k):
        if k == 1:
            return model.vy[k] == bird.vy + (model.u[k] + gravity) * dt
        else:
            return model.vy[k] == model.vy[k-1] + (model.u[k] + gravity) * dt

    model.dynamic_constraint_y = pyo.Constraint(model.k, rule=dynamic_constraint_y)
    model.dynamic_constraint_vy = pyo.Constraint(model.k, rule=dynamic_constraint_vy)

    # Objective function to minimize the distance to target and ensure smooth control
    model.objective = pyo.Objective(
        expr=sum((model.y[k] - model.T_target)**2 for k in model.k) + 0.1 * sum((model.u[k] - model.u[k-1])**2 for k in model.k if k > 1),
        sense=pyo.minimize)

    # Solve the optimization problem
    solver = pyo.SolverFactory('ipopt')
    solver.options['print_level'] = 0  # Suppress IPOPT output
    results = solver.solve(model, tee=False)

    # Extract the first control signal
    u_jump = pyo.value(model.u[1])
    return u_jump


   
# Bird variables and motion function
bird = Bird(50, 300, 30, 0)
x, y = transform(bird.x, bird.y)
bird_rect = pygame.Rect(x, y, bird.w, bird.h)

# Pipe variables and motion function
pipe_height = random.randint(200, 300)
pipe = Pipe(SCREEN_WIDTH - 50, pipe_height)

x, h = transform(pipe.x, pipe.h)
bottom_pipe_rect = pygame.Rect(x, 0, pipe.w, h)

x, y = transform(pipe.x, pipe.h + pipe.gap)
top_pipe_rect = pygame.Rect(x, y, pipe.w, SCREEN_HEIGHT - y)

# Clock
clock = pygame.time.Clock()
running = True
fps = 30
dt = 1 / fps

score = 0
while running:
    screen.fill(WHITE)

    # Handle events.
    u_jump = 0
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                u_jump = 500

    # Calculate the control signal
    u_jump = calculate_the_control_signal(bird, pipe)

    # Bird dynamics
    bird = bird_motion(bird, u_jump, dt)
    x, y = transform(bird.x, bird.y)
    bird_rect.y = y

    # Pipe dynamics
    pipe, d_score = pipe_motion(pipe, bird.vx, dt)
    x, y = transform(pipe.x, pipe.h)
    bottom_pipe_rect = pygame.Rect(x, y, pipe.w, pipe.h)
    top_pipe_rect = pygame.Rect(x, 0, pipe.w, SCREEN_HEIGHT - pipe.h - pipe.gap)

    # Update the score and bird velocity
    score += d_score
    bird.vx += d_score * 10

    # Draw bird and pipes
    pygame.draw.rect(screen, GREEN, bird_rect)
    pygame.draw.rect(screen, GREEN, bottom_pipe_rect)
    pygame.draw.rect(screen, GREEN, top_pipe_rect)

    # Draw the score
    font = pygame.font.Font(None, 36)
    text = font.render(f"Score: {score}", True, (0, 0, 0))
    screen.blit(text, (10, 10))

    # Collision detection
    if bird_rect.colliderect(bottom_pipe_rect) or \
            bird_rect.colliderect(top_pipe_rect) or \
            bird.y + bird.h > 1.5 * SCREEN_HEIGHT or \
            bird.y < -0.5 * SCREEN_HEIGHT:
        running = False

    # Update the display
    pygame.display.update()
    clock.tick(fps)

pygame.quit()

In [3]:
import copy
import pygame
import random
import pyomo.environ as pyo
from dataclasses import dataclass

@dataclass
class Bird:
    x: float
    y: float
    vx: float
    vy: float
    w: float = 20
    h: float = 20

@dataclass
class Pipe:
    x: float
    h: float
    w: float = 70
    gap: float = 200
    
    
def bird_motion(bird: Bird, u: float, dt: float, gravity: float = -50) -> Bird:
    """Updates the bird's y position and velocity."""
    new_bird = copy.deepcopy(bird)
    new_bird.y += new_bird.vy * dt
    new_bird.vy += (u + gravity) * dt
    return new_bird

def pipe_motion(pipe: Pipe, vx: float, dt: float, screen_width: int = 400) -> (Pipe, int):
    """Updates the pipe"""
    new_pipe = copy.deepcopy(pipe)
    new_pipe.x -= vx * dt

    d_score = 0
    if new_pipe.x < -new_pipe.w:
        new_pipe.x = screen_width
        new_pipe.h = random.randint(200, 300)
        d_score = 1
    return new_pipe, d_score